In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import MySQLdb
import numpy as np

In [2]:
db = MySQLdb.connect(host="localhost", user="root", passwd="root")
cursor = db.cursor()

In [3]:
infile = "../samples/macho/macho_vizier.tsv"
fdir = "/Users/Nick/Documents/AGBstuff/new_work/samples/macho"

In [4]:
def read_vizier(fin):
    f = open(fin).readlines()
    keep_lines = [line for line in f if (line.startswith("#") == False) & (line.startswith("\n") == False) ]
    columns = keep_lines[0][:-1].split("|")
    data = {}
    for ii in range(len(columns)):
        data[columns[ii]] = []
        
    for ii in range(3, len(keep_lines)):
        the_line = keep_lines[ii][:-1].split("|")
        for jj in range(len(columns)):
            value = the_line[jj]
            try:
                value = float(value)
                data[columns[jj]].append(value)
            except ValueError:
                value = value
                data[columns[jj]].append(value)
    
    data = pd.DataFrame(data)
    return data

In [5]:
def create_table():
    sql = "DROP TABLE agbtables.macho_raw;"
    cursor.execute(sql)
    
    sql = "CREATE TABLE agbtables.macho_raw (macho_cnt INT NOT NULL AUTO_INCREMENT,"
    sql += "field INT, tile INT, seq INT, ra DOUBLE, decl DOUBLE, glon DOUBLE, glat DOUBLE, per_r DOUBLE, per_b DOUBLE,"
    sql += "vmag DOUBLE, rmag DOUBLE, r_amp DOUBLE, b_amp DOUBLE, br_amp DOUBLE,"
    sql += "rsig DOUBLE, bsig DOUBLE, var_idx DOUBLE, star_class INT, n_obs INT,"
    sql += "PRIMARY KEY (macho_cnt))"
    cursor.execute(sql)

In [6]:
def ingest(fname):
    sql = "LOAD DATA LOCAL INFILE '%s'" % (fdir+"/"+fname)
    sql += " INTO TABLE agbtables.macho_raw FIELDS TERMINATED BY ',' ("
    sql += "field, tile, seq, ra, decl, glat, glon, per_r, per_b,"
    sql += "vmag, rmag, r_amp, b_amp, br_amp, rsig, bsig, var_idx, star_class, n_obs"
    sql += ");"
    
    sql_file = open(fdir+"/load_data.sql", "w")
    sql_file.write(sql)
    sql_file.close()
    os.system("mysql -u root --password=root < %s/load_data.sql" % fdir)

In [7]:
output = read_vizier(infile)

In [8]:
output.to_csv(fdir+"/macho_for_db.dat", sep=",", columns=["Field","Tile","Seqn","_RAJ2000","_DEJ2000","_Glat","_Glon","rPer","bPer","Vmag","Rmag","rAmp","bAmp","cAmp","rsig","bsig","Var","Class","Points"], header=False, index=False)

In [9]:
create_table()

In [10]:
ingest("macho_for_db.dat")

### Create a coordinate list of OGLE sources

In [59]:
def ogle_coordlist(fname):
    sql = "SELECT ra, decl FROM agbtables.ogle3_agbs WHERE ra > 0"
    cursor.execute(sql)
    output = cursor.fetchall()
    
    fmt = "%f\t%f\n"
    fout = open(fdir+"/"+fname, "w")
    for ii in range(len(output)):
        fout.write(fmt % output[ii])
        
    fout.close()

In [172]:
def read_vizier_matched(fin):
    f = open(fin).readlines()
    keep_lines = [line for line in f if line.startswith("#") == False]
    columns = keep_lines[1][:-1].split("|")
    data = {}
    data["ogleid"] = []
    for ii in range(len(columns)):
        data[columns[ii]] = []
    
    for ii in range(4, len(keep_lines)):
        data["ogleid"].append(ii-3)
        if keep_lines[ii].startswith("\n"):
            for jj in range(len(columns)):
                data[columns[jj]].append("")
        else:        
            the_line = keep_lines[ii][:-1].split("|")
            for jj in range(len(columns)):
                value = the_line[jj]
                try:
                    value = float(value)
                    data[columns[jj]].append(value)
                except ValueError:
                    value = value
                    data[columns[jj]].append(value)
    
    data = pd.DataFrame(data)
    return data

In [60]:
ogle_coordlist("ogle_coordlist.dat")

In [146]:
matchfile = fdir + "/" + "ogle_macho_match2.tsv"

In [173]:
matched = read_vizier_matched(matchfile)

In [200]:
def create_match_table():
    sql = "DROP TABLE agbtables.macho_ogle_match;"
    cursor.execute(sql)
    
    sql = "CREATE TABLE agbtables.macho_ogle_match (match_cnt INT NOT NULL AUTO_INCREMENT,"
    sql += "field INT, tile INT, seq INT, ra DOUBLE, decl DOUBLE, glon DOUBLE, glat DOUBLE, per_r DOUBLE, per_b DOUBLE,"
    sql += "vmag DOUBLE, rmag DOUBLE, r_amp DOUBLE, b_amp DOUBLE, br_amp DOUBLE,"
    sql += "rsig DOUBLE, bsig DOUBLE, var_idx DOUBLE, star_class INT, n_obs INT, matchrad DOUBLE, ogleid INT,"
    sql += "PRIMARY KEY (match_cnt))"

    cursor.execute(sql)

In [181]:
def ingest_match(fname):
    sql = "LOAD DATA LOCAL INFILE '%s'" % (fdir+"/"+fname)
    sql += " INTO TABLE agbtables.macho_ogle_match FIELDS TERMINATED BY ',' ("
    sql += "field, tile, seq, ra, decl, glat, glon, per_r, per_b,"
    sql += "vmag, rmag, r_amp, b_amp, br_amp, rsig, bsig, var_idx, star_class, n_obs, matchrad, ogleid"
    sql += ");"
    
    sql_file = open(fdir+"/load_data.sql", "w")
    sql_file.write(sql)
    sql_file.close()
    os.system("mysql -u root --password=root < %s/load_data.sql" % fdir)

In [182]:
matched.to_csv(fdir+"/matched_for_db.dat", sep=",", columns=["Field","Tile","Seqn","_RAJ2000","_DEJ2000","_Glat","_Glon","rPer","bPer","Vmag","Rmag","rAmp","bAmp","cAmp","rsig","bsig","Var","Class","Points","_r","ogleid"], header=False, index=False)

In [199]:
create_match_table()
ingest_match("matched_for_db.dat")

In [202]:
def create_match_table_alldata():
#     sql = "DROP TABLE agbtables.macho_ogle_match_alldata;"
#     cursor.execute(sql)
    
    sql = "CREATE TABLE agbtables.macho_ogle_match_alldata AS (SELECT "
    sql += "m.field, m.tile, m.seq, m.ra, m.decl, m.glat, m.glon, m.per_r, m.per_b,"
    sql += "m.vmag, m.rmag, m.r_amp, m.b_amp, m.br_amp, m.rsig, m.bsig, m.var_idx, m.star_class,"
    sql += "m.n_obs, m.matchrad, o.p, o.I_mean, o.V_mean, o.I_amp, o.type, o.evol, o.spectr "
    sql += "FROM ogle3_agbs AS o INNER JOIN macho_ogle_match AS m ON o.ogle3cnt = m.ogleid "
    sql += "WHERE m.n_obs != 0"
    sql += ")"

    print sql 
    cursor.execute(sql)

In [203]:
create_match_table_alldata()

CREATE TABLE agbtables.macho_ogle_match_alldata AS (SELECT m.field, m.tile, m.seq, m.ra, m.decl, m.glat, m.glon, m.per_r, m.per_b,m.vmag, m.rmag, m.r_amp, m.b_amp, m.br_amp, m.rsig, m.bsig, m.var_idx, m.star_class,m.n_obs, m.matchrad, o.p, o.I_mean, o.V_mean, o.I_amp, o.type, o.evol, o.spectr FROM ogle3_agbs AS o INNER JOIN macho_ogle_match AS m ON o.ogle3cnt = m.ogleid WHERE m.n_obs != 0)


OperationalError: (1046, 'No database selected')